In [1]:
import numpy as np
import torch
import torchvision.transforms as transforms
from torch import nn, optim
from PIL import Image

In [2]:
image = Image.open("gambar-prediksi.jpg").convert("RGB")

In [3]:
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(5),
    transforms.ToTensor(),
    transforms.Normalize((0.488,), (0.2172,))
])

transformed_image = transform(image)

print(transformed_image)
transformed_image_bulat = np.ceil(transformed_image)
X = torch.tensor(np.where(transformed_image_bulat == -0, 0, transformed_image_bulat))
X = X.unsqueeze(0)
print(X)

tensor([[[ 1.9962,  1.9962,  0.0823, -0.7121, -0.2968],
         [ 1.0393,  1.3101, -0.3690, -0.8204, -0.6038],
         [ 1.6892,  1.3462, -0.2607, -1.0732, -1.0010],
         [ 1.6892,  1.5448,  0.3893, -0.8565, -0.9468],
         [ 1.0573,  0.7323,  0.2268, -0.4413, -0.7121]],

        [[-0.2968,  0.2268, -0.0441,  0.2990,  0.6240],
         [ 0.2990,  1.2198,  0.4976,  0.3893, -0.0079],
         [ 1.8698,  1.5990,  0.7323,  0.1546, -0.1704],
         [ 1.8698,  1.6712,  1.0573,  0.4073,  0.3532],
         [ 1.2740,  1.2198,  1.1656,  0.7143,  0.3893]],

        [[ 0.5518,  0.9851,  0.7684,  1.0212,  1.2920],
         [ 1.2559,  1.7976,  1.2740,  1.2017,  0.9490],
         [ 2.2489,  2.0864,  1.4906,  1.0393,  0.8045],
         [ 2.2851,  2.2128,  1.7434,  1.2017,  1.1837],
         [ 2.0684,  1.9781,  1.8156,  1.4545,  1.2920]]])
tensor([[[[ 2.,  2.,  1.,  0.,  0.],
          [ 2.,  2.,  0.,  0.,  0.],
          [ 2.,  2.,  0., -1., -1.],
          [ 2.,  2.,  1.,  0.,  0.],
      

In [4]:
print(X.size())

torch.Size([1, 3, 5, 5])


In [187]:
class MYEfficientNetV2(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=3,
                            out_channels=1,
                            kernel_size=3,
                            bias=False)
    self.conv1.weight = nn.Parameter(torch.tensor(
        [[[[0, -1, 1],
           [1, 2, -1],
           [2, -1, -2]],
          [[2, -1, 0],
           [1, 0, -2],
           [0, -1, -1]],
          [[-1, 0, -1],
           [1, -1, 2],
           [1, 1, -1]]]], dtype = torch.float32, requires_grad=True))
    self.batchnorm = nn.BatchNorm2d(1, affine=True)
    self.activation = nn.SiLU()
    self.pointwise_conv = nn.Conv2d(
            in_channels = 1,
            out_channels = 3,
            kernel_size=1,
            bias=False,
        )
    self.pointwise_conv.weight = nn.Parameter(torch.tensor(
        [[[[0.5]]],
           [[[0.2]]],
           [[[0.8]]]], dtype = torch.float32, requires_grad=True))

    self.batchnorm2 = nn.BatchNorm2d(3, affine=True)

    self.depthwise = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, padding=1, padding_mode='zeros',
                                   groups=3, bias=False)
    
    self.depthwise.weight = nn.Parameter(torch.tensor(
            [[[[0, -1, 1], [1, 2, -1], [2, -1, -2]]],
             [[[2, -1, 0], [1, 0, -2], [0, -1, -1]]],
             [[[-1, 0, -1], [1, -1, 2], [1, 1, -1]]]], dtype=torch.float32, requires_grad=True))
    
    self.seblock = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(3, 3, bias=False),
            nn.Sigmoid()
        )
    mat2 = torch.tensor([[0.2089591267, 0.2089591267, 0.2089591267],
                        [0.2096759506, 0.2096759506, 0.2096759506],
                        [0.2117456203, 0.2117456203, 0.2117456203]], dtype=torch.float32)
    self.seblock[2].weight.data = nn.Parameter(mat2)

    self.fc = nn.Linear(in_features=27,
                        out_features=3,
                        bias=False)

    self.fc.weight = nn.Parameter(torch.tensor(
        [[0, -1, 1, 1, 2, -1, 2, -1, -2],
         [2, -1, 0, 1, 0, -2, 0, -1, -1],
         [-1, 0, -1, 1, -1, 2, 1, 1, -1]], dtype = torch.float32, requires_grad=True))
    
    self.softmax = nn.Softmax(dim=1)

  def forward(self, X):
    x = self.conv1(X)
    print("Convolusi:\n", x)
    x = self.batchnorm(x)
    print("Batch Normalization:\n", x)
    x = self.activation(x)
    print("Swish:\n", x)

    x = self.pointwise_conv(x)
    print("Pointwise Convolution:\n", x)
    x = self.batchnorm2(x)
    print("Pointwise Convolution Batch Normalization:\n", x)
    x = self.activation(x)
    print("Pointwise Convolution Swish:\n", x)
    x = self.depthwise(x)
    print("Depthwise:\n", x)
    x = self.batchnorm2(x)
    print("Depthwise Batch Normalization:\n", x)
    x = self.activation(x)
    print("Depthwise Swish:\n", x)
    se_weights = self.seblock(x)
    x = x * se_weights[:, :, None, None]
    print("SEBlok:\n", x)
    
    hasil = torch.flatten(x)
    print("Flatten:\n", hasil)
    x = self.fc(x.view(3, 9))
    print("Result Neural Network:\n", x)
    x = self.softmax(x)
    print("Softmax:\n", x)
    return x


In [196]:
model = MYEfficientNetV2()
optimizer = optim.SGD(model.parameters(), lr=0.001)
output = model(X)

Convolusi:
 tensor([[[[ 8., 11.,  4.],
          [ 4., 10.,  2.],
          [ 2.,  6.,  5.]]]], grad_fn=<ConvolutionBackward0>)
Batch Normalization:
 tensor([[[[ 0.7207,  1.6938, -0.5766],
          [-0.5766,  1.3694, -1.2253],
          [-1.2253,  0.0721, -0.2523]]]], grad_fn=<NativeBatchNormBackward0>)
Swish:
 tensor([[[[ 0.4849,  1.4308, -0.2074],
          [-0.2074,  1.0918, -0.2781],
          [-0.2781,  0.0373, -0.1103]]]], grad_fn=<SiluBackward0>)
Pointwise Convolution:
 tensor([[[[ 0.2425,  0.7154, -0.1037],
          [-0.1037,  0.5459, -0.1391],
          [-0.1391,  0.0187, -0.0552]],

         [[ 0.0970,  0.2862, -0.0415],
          [-0.0415,  0.2184, -0.0556],
          [-0.0556,  0.0075, -0.0221]],

         [[ 0.3879,  1.1446, -0.1659],
          [-0.1659,  0.8735, -0.2225],
          [-0.2225,  0.0299, -0.0882]]]], grad_fn=<ConvolutionBackward0>)
Pointwise Convolution Batch Normalization:
 tensor([[[[ 0.4405,  2.0023, -0.7027],
          [-0.7027,  1.4427, -0.8195],
     

In [202]:
criterion = nn.CrossEntropyLoss()
target = torch.tensor([[1, 0, 0],
                       [0, 0, 1],
                       [0, 1, 0]])
loss = criterion(output, torch.argmax(target, dim=1))
print("Loss:\n", loss)

Loss:
 tensor(0.7580, grad_fn=<NllLossBackward0>)


In [ ]:
loss.backward()
optimizer.step()
print("Fully Connected grad:\n", model.fc.weight.grad)
print("new weight FC:\n", model.fc.weight)

Fully Connected grad:
 tensor([[ 0.0441, -0.0455,  0.0115,  0.0088, -0.0111,  0.0093, -0.0147,  0.0088,
          0.0049],
        [-0.0466,  0.0248, -0.0420, -0.0138,  0.0106,  0.0159,  0.0132, -0.0062,
          0.0095],
        [ 0.0025,  0.0207,  0.0304,  0.0050,  0.0005, -0.0252,  0.0014, -0.0026,
         -0.0144]])
new weight FC:
 Parameter containing:
tensor([[-4.4088e-05, -9.9995e-01,  9.9999e-01,  9.9999e-01,  2.0000e+00,
         -1.0000e+00,  2.0000e+00, -1.0000e+00, -2.0000e+00],
        [ 2.0000e+00, -1.0000e+00,  4.1965e-05,  1.0000e+00, -1.0576e-05,
         -2.0000e+00, -1.3238e-05, -9.9999e-01, -1.0000e+00],
        [-1.0000e+00, -2.0735e-05, -1.0000e+00,  1.0000e+00, -1.0000e+00,
          2.0000e+00,  1.0000e+00,  1.0000e+00, -9.9999e-01]],
       requires_grad=True)
